In [116]:
def is_variable(pat):
    return pat.startswith('?') and all(s.isalpha() for s in pat[1:])

In [117]:
def pat_match(patten,saying):
    if is_variable(patten[0]):
        print (patten[0]) 
        return True
    
    else:
        if patten[0] != saying[0]: return True
        else:
            return pat_match(patten[1:],saying[1:])

In [118]:
pat_match("I want to learn ?X ".split(),"I want to learn NLP".split())

?X


True

In [119]:
pat_match ("It is a ?X".split(),"It is a dog".split())

?X


True

# 修改优化，获得每个variable对应的是什么值

In [120]:
def pat_match(pattern,saying):
    if is_variable(pattern[0]):
        return pattern[0],saying[0]
    else:
        if pattern[0] !=saying[0]:return False
        else:
            return pat_match(pattern[1:],saying[1:])

In [121]:
pat_match("I want to have ?X".split(),"I want to have house".split())

('?X', 'house')

In [122]:
pat_match("?X for ?X".split(),"AI  for NLP".split())

('?X', 'AI')

# 进一步优化,匹配多个变量

In [123]:
def pat_match(pattern,saying):
    if not  pattern or not  saying:return []
    
    if is_variable(pattern[0]):
        print(pattern[0])
        return [(pattern[0],saying[0])] + pat_match(pattern[1:],saying[1:])
    
    else:
        if pattern[0] != saying[0]:return []
        else:
            return pat_match(pattern[1:],saying[1:])

In [124]:
pat_match("?X greater ?W than ?Y ?Z".split(),"3 greater 4 than 2".split())

?X
?W
?Y


[('?X', '3'), ('?W', '4'), ('?Y', '2')]

In [125]:
def pat_to_dic(patterns):
    return {k:v for k,v in patterns}

In [126]:
def subsititle(rule,parsed_rules):
    if not rule: return []
    
    return [parsed_rules.get(rule[0],rule[0])] +subsititle(rule[1:],parsed_rules)

In [127]:
got_patterns =pat_match("I want to buy ?X".split()," I want to buy iPhone".split())

?X


In [128]:
got_patterns

[('?X', 'iPhone')]

In [129]:
subsititle("what if you mean if you got a ?X".split(),pat_to_dic(got_patterns))

['what', 'if', 'you', 'mean', 'if', 'you', 'got', 'a', 'iPhone']

In [130]:
' '.join(subsititle("what if you mean if you got a ?X".split(),pat_to_dic(got_patterns)))

'what if you mean if you got a iPhone'

In [131]:
pat_to_dic(got_patterns)

{'?X': 'iPhone'}

In [132]:
join_pat = pat_match("?p need ?X".split(),"iPhone need set".split())

?p
?X


In [133]:
join_pat

[('?p', 'iPhone'), ('?X', 'set')]

In [134]:
subsititle("why does ?p need ?X ?".split(),pat_to_dic(join_pat))

['why', 'does', 'iPhone', 'need', 'set', '?']

In [135]:
' '.join(subsititle("why does ?p need ?X ?".split(),pat_to_dic(join_pat)))

'why does iPhone need set ?'

In [136]:
defined_patterns = {
     "I need ?X": ["Image you will get ?X soon", "Why do you need ?X ?"], 
     "My ?X told me something": ["Talk about more about your ?X", 
                                "How do you think about your ?X ?"]
}

In [137]:
pattern_1 = pat_match("I need ?X".split(),"I need iPhone".split())

?X


In [138]:
' '.join(subsititle("Image you will get ?X soon".split(),pat_to_dic(pattern_1)))

'Image you will get iPhone soon'

In [139]:
import random

In [140]:
defined_patterns

{'I need ?X': ['Image you will get ?X soon', 'Why do you need ?X ?'],
 'My ?X told me something': ['Talk about more about your ?X',
  'How do you think about your ?X ?']}

有点理解，但是没有get到那个main idea .

In [141]:
def get_response(saying,rules):
    for key in rules:
        if pat_match(key.split(),saying.split()):
            rule = pat_match(key.split(),saying.split())
            pattern = (random.choice(rules[key]))
            return ' '.join(subsititle(pattern.split(),pat_to_dic(rule)))


In [142]:
get_response("I need iPhone",defined_patterns)

?X
?X


'Image you will get iPhone soon'

In [143]:
get_response("My mother told me something",defined_patterns)

?X
?X


'How do you think about your mother ?'

In [144]:
 def is_variable_segment(pattern):
    return pattern.startswith("?*") and all( a.isalpha for a in pattern[2:])

In [145]:
is_variable_segment("?*aaa")

True

In [146]:
from collections import defaultdict

In [147]:
fail = [True, None]

def pat_match_with_seg(pattern,saying):
    if not pattern or not saying: return []

    pat = pattern[0]
    
    if is_variable(pat):
        return [(pat,saying[0]) + pat_match_with_seg(pattern[1:],saying[1:])]
    elif is_variable_segment(pat):
        match,index = segment_match(pattern,saying)
        if pattern[1:] and not saying[index:]:
            return fail
        return [match] + pat_match_with_seg(pattern[1:],saying[index:])
    elif pat == saying[0]:
        if pattern[1:] and not saying[1:]:
            return fail
        return pat_match_with_seg(pattern[1:],saying[1:])
    else:
        return fail

In [148]:
def segment_match(pattern, saying):
    seg_pat,rest = pattern[0],pattern[1:]
    seg_pat = seg_pat.replace('?*','?')
    
    if not rest:return (seg_pat,saying),len(saying)
    
    for i,token in enumerate(saying):
        if rest[0] == token :
            return (seg_pat,saying[:i]), i
    return (seg_pat,saying),len(saying)
            

In [149]:
segment_match("?*P is very good".split(),"My computer and My iPhone is very good".split())

(('?P', ['My', 'computer', 'and', 'My', 'iPhone']), 5)

In [150]:
pat_match_with_seg("?*P is very good and ?*X".split(),"My dog is very good and my cat is very cut".split())

[('?P', ['My', 'dog']), ('?X', ['my', 'cat', 'is', 'very', 'cut'])]

In [151]:
pat_match_with_seg(" I need an  ?*X".split(),"I need an iPhone".split())

[('?X', ['iPhone'])]

In [152]:
def pat_to_dict(patterns):
    return {k:' '.join(v) if isinstance(v,list) else v for k,v in patterns}

In [153]:
sub = subsititle("Why do you need ?X".split(),pat_to_dict(pat_match_with_seg('I need ?*X'.split(),'I need an iPhone'.split())))

In [154]:
' '.join(sub)

'Why do you need an iPhone'

In [155]:
rules = {
    "?*X hello ?*Y": ["Hi, how do you do?"],
    "I was ?*X": ["Were you really ?X ?", "I already knew you were ?X ."],
    "?*P is very good and ?*X":["My dog is very good and my cat is very cut"]
}

In [156]:
rules

{'?*X hello ?*Y': ['Hi, how do you do?'],
 'I was ?*X': ['Were you really ?X ?', 'I already knew you were ?X .'],
 '?*P is very good and ?*X': ['My dog is very good and my cat is very cut']}

有些细节没有想通，还没有写出来

In [157]:
def get_reponse_seg(saying,rules):
    for key,value in rules.items():
            rule = pat_match_with_seg(key.split(),saying.split())
            for i in rule:
                if i == None:
                    rule == fail
                    continue
            if rule != fail:
                return ' '.join(subsititle(random.choice(value).split(),pat_to_dict(rule)))


In [158]:
get_reponse_seg("He is very good and He is very tall",rules)

'My dog is very good and my cat is very cut'

In [159]:
pat_match_with_seg("?*X hello ?*Y".split(),"you hello me".split())

[('?X', ['you']), ('?Y', ['me'])]

In [160]:
get_reponse_seg("you hello me",rules)

'Hi, how do you do?'

In [161]:
import jieba

In [162]:
def get_chinses_response(saying,rules):
    for r in rules:
        if match_rule(r.split(),saying.split()):
            return ' '.join(subsititle(random.choice(rules[key]),pat_to_dict(pat_match_with_seg(saying.split()))))

In [163]:
def match_chinese_rule(rule,saying):
    words = [r for r in rule if r.isalpha()]
    if not words: return True
    for w in words:
        if not w in saying:return False
    return True
    

In [164]:
match_rule("i need ?*?"," i need iPhone")

True

In [166]:
rule_responses = {
    '?*x hello ?*y': ['How do you do', 'Please state your problem'],
    '?*x I want ?*y': ['what would it mean if you got ?y', 'Why do you want ?y', 'Suppose you got ?y soon'],
    '?*x if ?*y': ['Do you really think its likely that ?y', 'Do you wish that ?y', 'What do you think about ?y', 'Really-- if ?y'],
    '?*x no ?*y': ['why not?', 'You are being a negative', 'Are you saying \'No\' just to be negative?'],
    '?*x I was ?*y': ['Were you really', 'Perhaps I already knew you were ?y', 'Why do you tell me you were ?y now?'],
    '?*x I feel ?*y': ['Do you often feel ?y ?', 'What other feelings do you have?'],
    '?*x你好?*y': ['你好呀', '请告诉我你的问题'],
    '?*x我想?*y': ['你觉得?y有什么意义呢？', '为什么你想?y', '你可以想想你很快就可以?y了'],
    '?*x我想要?*y': ['?x想问你，你觉得?y有什么意义呢?', '为什么你想?y', '?x觉得... 你可以想想你很快就可以有?y了', '你看?x像?y不', '我看你就像?y'],
    '?*x喜欢?*y': ['喜欢?y的哪里？', '?y有什么好的呢？', '你想要?y吗？'],
    '?*x讨厌?*y': ['?y怎么会那么讨厌呢?', '讨厌?y的哪里？', '?y有什么不好呢？', '你不想要?y吗？'],
    '?*xAI?*y': ['你为什么要提AI的事情？', '你为什么觉得AI要解决你的问题？'],
    '?*x机器人?*y': ['你为什么要提机器人的事情？', '你为什么觉得机器人要解决你的问题？'],
    '?*x对不起?*y': ['不用道歉', '你为什么觉得你需要道歉呢?'],
    '?*x我记得?*y': ['你经常会想起这个吗？', '除了?y你还会想起什么吗？', '你为什么和我提起?y'],
    '?*x如果?*y': ['你真的觉得?y会发生吗？', '你希望?y吗?', '真的吗？如果?y的话', '关于?y你怎么想？'],
    '?*x我?*z梦见?*y':['真的吗? --- ?y', '你在醒着的时候，以前想象过?y吗？', '你以前梦见过?y吗'],
    '?*x妈妈?*y': ['你家里除了?y还有谁?', '嗯嗯，多说一点和你家里有关系的', '她对你影响很大吗？'],
    '?*x爸爸?*y': ['你家里除了?y还有谁?', '嗯嗯，多说一点和你家里有关系的', '他对你影响很大吗？', '每当你想起你爸爸的时候， 你还会想起其他的吗?'],
    '?*x我愿意?*y': ['我可以帮你?y吗？', '你可以解释一下，为什么想?y'],
    '?*x我很难过，因为?*y': ['我听到你这么说， 也很难过', '?y不应该让你这么难过的'],
    '?*x难过?*y': ['我听到你这么说， 也很难过',
                 '不应该让你这么难过的，你觉得你拥有什么，就会不难过?',
                 '你觉得事情变成什么样，你就不难过了?'],
    '?*x就像?*y': ['你觉得?x和?y有什么相似性？', '?x和?y真的有关系吗？', '怎么说？'],
    '?*x和?*y都?*z': ['你觉得?z有什么问题吗?', '?z会对你有什么影响呢?'],
    '?*x和?*y一样?*z': ['你觉得?z有什么问题吗?', '?z会对你有什么影响呢?'],
    '?*x我是?*y': ['真的吗？', '?x想告诉你，或许我早就知道你是?y', '你为什么现在才告诉我你是?y'],
    '?*x我是?*y吗': ['如果你是?y会怎么样呢？', '你觉得你是?y吗', '如果你是?y，那一位着什么?'],
    '?*x你是?*y吗':  ['你为什么会对我是不是?y感兴趣?', '那你希望我是?y吗', '你要是喜欢， 我就会是?y'],
    '?*x你是?*y' : ['为什么你觉得我是?y'],
    '?*x因为?*y' : ['?y是真正的原因吗？', '你觉得会有其他原因吗?'],
    '?*x我不能?*y': ['你或许现在就能?*y', '如果你能?*y,会怎样呢？'],
    '?*x我觉得?*y': ['你经常这样感觉吗？', '除了到这个，你还有什么其他的感觉吗？'],
    '?*x我?*y你?*z': ['其实很有可能我们互相?y'],
    '?*x你为什么不?*y': ['你自己为什么不?y', '你觉得我不会?y', '等我心情好了，我就?y'],
    '?*x好的?*y': ['好的', '你是一个很正能量的人'],
    '?*x嗯嗯?*y': ['好的', '你是一个很正能量的人'],
    '?*x不嘛?*y': ['为什么不？', '你有一点负能量', '你说 不，是想表达不想的意思吗？'],
    '?*x不要?*y': ['为什么不？', '你有一点负能量', '你说 不，是想表达不想的意思吗？'],
    '?*x有些人?*y': ['具体是哪些人呢?'],
    '?*x有的人?*y': ['具体是哪些人呢?'],
    '?*x某些人?*y': ['具体是哪些人呢?'],
    '?*x每个人?*y': ['我确定不是人人都是', '你能想到一点特殊情况吗？', '例如谁？', '你看到的其实只是一小部分人'],
    '?*x所有人?*y': ['我确定不是人人都是', '你能想到一点特殊情况吗？', '例如谁？', '你看到的其实只是一小部分人'],
    '?*x总是?*y': ['你能想到一些其他情况吗?', '例如什么时候?', '你具体是说哪一次？', '真的---总是吗？'],
    '?*x一直?*y': ['你能想到一些其他情况吗?', '例如什么时候?', '你具体是说哪一次？', '真的---总是吗？'],
    '?*x或许?*y': ['你看起来不太确定'],
    '?*x可能?*y': ['你看起来不太确定'],
    '?*x他们是?*y吗？': ['你觉得他们可能不是?y？'],
    '?*x': ['很有趣', '请继续', '我不太确定我很理解你说的, 能稍微详细解释一下吗?']
}

In [167]:
get_response("你总是run",rule_responses)

In [168]:
!pwd

/Users/tom/Desktop/AI-For-NLP/Assignment-01


In [169]:
def get_response(saying, rules):
    for r in rules:
        if match_rule(r.split(),saying.split()):
            return subsititle(random.choice(rules[r]).split(),pat_to_dict(pat_match_with_seg(r.split(),saying.split())))

In [170]:
def match_rule(rule,saying):
    words = [r for r in rule if r.isalpha()]
    if not words: return True
    for w in words:
        if not w in saying:
            return False
    return True

In [171]:
from  icecream import ic 

In [178]:
def get_response_chinese(saying ,rules):
    for r in rules:
        if match_rule_chinese(search_charater(r).split(),search_charater(saying).split()):
            return subsititle(search_charater_seg(random.choice(rules[r])).split(),
                             pat_to_dict(pat_match_with_seg(search_charater(r).split(), search_charater(saying).split())))                            

In [213]:
def match_rule_chinese(rule,saying):
    words = [r for r in rule if not r.startswith('?')]
    if not words: return []
    for w in words:
        if  not w  in saying:
            return False
    return True


In [219]:
def search_charater(rule):
    if not rule: return []
    word_list = [a for i,a in enumerate(jieba.cut(rule))]
    if word_list[0] == '?':
        r = rule[:3]
    else:
        r =word_list[0]
    return ' '.join(s for s in (r,''.join(search_charater(rule[len(r):]))))

In [220]:
def search_charater_seg(rule):
    if not rule: return []
    word_list = [a for i,a in enumerate(jieba.cut(rule))]
    if word_list[0] == '?':
        r = rule[:2]
    else:
        r = word_list[0]
    return ''.join(s for s in (r,''.join(search_charater_seg(rule[len(r):]))))

In [223]:
' '.join(get_response_chinese("你妈妈很好",rule_responses))

'嗯嗯，多说一点和你家里有关系的'

In [222]:
' '.join(get_response_chinese("他一直很好",rule_responses))

'你能想到一些其他情况吗?'

1.这样的程序有什么优点，有什么缺点，你有什么可以改进的方法？
  刚刚接触，很多东西不是很理解，这个也是参考别人的，才能看懂，希望随着学习的深入，能够看出它的有优缺点。
2.什么是数据驱动？数据驱动在这里程序里如何体现？
  个人觉得，一切都可以数据，毕竟很多东西都必须数据化，才能编程，才能让计算机更好的理解人类的世界。
3.数据驱动和AI的关系是什么？
  AI的发展必须依靠数据的驱动，数据是AI的粮食，没有大量的数据，AI无法学习，无法发展，无法演化。